In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
from transformers import GPT2Tokenizer,GPT2Model,AutoModel

from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset

from tokenizers import Tokenizer

[2023-08-04 22:20:39,707] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
#然后我们可以使用from_file() 方法从该文件里重新加载 Tokenizer 对象：
new_tokenizer = Tokenizer.from_file("human2_formal.json")
#或者下面方法
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)

In [3]:
#model = GPT2LMHeadModel.from_pretrained("gpt2")
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=128,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)

In [8]:
train_file = "human2.fna.line.train"
eval_file = "human2.fna.line.valid"
max_seq_length = 512
out_model_path = "mygpt_formal"
train_epoches = 10
batch_size = 20

tokenizer.pad_token = tokenizer.eos_token

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=max_seq_length,
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=eval_file,
    block_size=max_seq_length,
)

training_args = TrainingArguments(
        output_dir=out_model_path,
        overwrite_output_dir=True,
        num_train_epochs=train_epoches,
        per_device_train_batch_size=batch_size,
        save_steps=2000,
        save_total_limit=2,
        prediction_loss_only=True,
        #fp16=True, v100没法用
    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

In [9]:
trainer.train()
trainer.save_model(out_model_path)

Step,Training Loss
500,8.359400
1000,8.273700
1500,8.233900
2000,8.190600
2500,8.152000
3000,8.103200
3500,8.050300
4000,8.003500
4500,7.968600
5000,7.928400


In [12]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 4754.71
